In [1]:
import sys
sys.path.append("models/")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from setup import *
from dataloader import SurveyDataset
import mnl

%load_ext autoreload
%autoreload 2

In [2]:
tp = pd.read_csv(data_dir+"trips.csv")
n_alts = 4

In [3]:
print(tp['mode'].value_counts()/len(tp))

2    0.713060
1    0.132001
4    0.111893
3    0.043046
Name: mode, dtype: float64


In [4]:
tp['morning'] = (tp['dep_hour'] > 6) & (tp['dep_hour'] < 10)
tp['afternoon'] = (tp['dep_hour'] > 15) & (tp['dep_hour'] < 19)
tp['morning'] = tp['morning'].astype(int)
tp['afternoon'] = tp['afternoon'].astype(int)

def normalize_features(df, cols):
    for c in cols:
        df[c] = df[c]/df[c].max()
    return df

In [5]:
tp['const'] = 1

In [6]:
x = tp[['const','morning','afternoon','companion', 'distance', 
         'from_home', 'to_home', 'purp_work', 'purp_school', 'purp_errand', 'purp_recreation', 
         'ontime_important', '12_18yrs', '18_25yrs', '25_55yrs', '55+yrs', 'no_age', 
         'disability', 'educ_col', 'educ_grad', 
         'race_white', 'race_black', 'race_asian', 
         'male', 'female', 
         'emply_park', 'emply_transit', 'emply_veh', 'emply_wfh', 'emply_flex', 'emply_hours', 
         'license', 'person_trips', 'person_transit', 'person_freq_transit', 
         'hh_inc_0_30', 'hh_inc_30_60', 'hh_inc_60_100', 'hh_inc_100_150', 'hh_inc_150', 
         'avg_pr_veh', 'home_own', 'home_house', 'home_condo']].to_numpy()

x = x[:, np.newaxis, :]
x = np.tile(x, (1, n_alts, 1))

y = tp['mode'].astype(int).to_numpy() - 1
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [7]:
x.shape

(84143, 4, 44)

In [7]:
trainset = SurveyDataset(torch.tensor(x_train), torch.tensor(y_train, dtype=torch.long))
trainloader = DataLoader(trainset, batch_size=256, shuffle=True)

testset = SurveyDataset(torch.tensor(x_test), torch.tensor(y_test, dtype=torch.long))
testloader = DataLoader(testset, batch_size=len(testset), shuffle=True)

In [8]:
loss_fn = nn.CrossEntropyLoss()
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = mnl.MNL(n_alts=n_alts, n_features=x.shape[-1])
# model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=0)

In [9]:
for epoch in range(10):
    loss_ = 0
    correct = 0
    for batch, (x_batch, y_batch) in enumerate(trainloader):
        # Compute prediction and loss
        util = model(x_batch)
        loss = loss_fn(util, y_batch)
        loss_ += loss.item()
        
        pred = torch.argmax(util, dim=1)
        correct += torch.sum(pred == y_batch)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    if batch % 1 == 0:
        print(f"[epoch: {epoch:>3d}] Train loss: {loss_/len(trainset):.4f} accuracy: {correct/len(trainset):.3f}")
    
    
    correct = 0
    for batch, (x_batch, y_batch) in enumerate(testloader):
        util = model(x_batch)
        pred = torch.argmax(util, dim=1)
        correct += torch.sum(pred == y_batch)
    print(f"[epoch: {epoch:>3d}] Test accuracy: {correct/len(testset):.3f}")


[epoch:   0] Train loss: 0.0035 accuracy: 0.783
[epoch:   0] Test accuracy: 0.832
[epoch:   1] Train loss: 0.0019 accuracy: 0.833
[epoch:   1] Test accuracy: 0.837
[epoch:   2] Train loss: 0.0018 accuracy: 0.839
[epoch:   2] Test accuracy: 0.839
[epoch:   3] Train loss: 0.0018 accuracy: 0.839
[epoch:   3] Test accuracy: 0.837
[epoch:   4] Train loss: 0.0018 accuracy: 0.840
[epoch:   4] Test accuracy: 0.839
[epoch:   5] Train loss: 0.0018 accuracy: 0.841
[epoch:   5] Test accuracy: 0.840
[epoch:   6] Train loss: 0.0018 accuracy: 0.840
[epoch:   6] Test accuracy: 0.842
[epoch:   7] Train loss: 0.0018 accuracy: 0.841
[epoch:   7] Test accuracy: 0.843
[epoch:   8] Train loss: 0.0018 accuracy: 0.841
[epoch:   8] Test accuracy: 0.832
[epoch:   9] Train loss: 0.0018 accuracy: 0.840
[epoch:   9] Test accuracy: 0.842
